### Part 3 Silver+-+Traffic+Transformations

In [ ]:
checkpoint = spark.sql("describe external location `checkpoints`").select("url").collect()[0][0]
bronze = spark.sql("describe external location `bronze`").select("url").collect()[0][0]
silver = spark.sql("describe external location `silver`").select("url").collect()[0][0]

In [ ]:
%run "/Workspace/Users/sachinsax_gmail.com#ext#@sachinsaxgmail.onmicrosoft.com/Part 4 Common"

In [ ]:
print(checkpoint)
print(bronze)
print(silver)

abfss://checkpoints@databricksdevstg123.dfs.core.windows.net/
abfss://medallion@databricksdevstg123.dfs.core.windows.net/bronze
abfss://medallion@databricksdevstg123.dfs.core.windows.net/silver


In [ ]:
dbutils.widgets.text(name="env",defaultValue='',label='Enter the environment in lower case')
env = dbutils.widgets.get("env")

### Reading the data from bronze Table

In [ ]:
def read_BronzeTrafficTable(environment):
    print('Reading the Bronze Table Data : ',end='')
    df_bronzeTraffic = (spark.readStream
                    .table(f"`{environment}-catalog`.`bronze`.raw_traffic")
                    )
    print(f'Reading {environment}_catalog.bronze.raw_traffic Success!')
    return df_bronzeTraffic

### Handing duplicate rows

In [ ]:
def remove_Dups(df):
    print('Removing Duplicate values: ', end='')
    df_dup = df.dropDuplicates()
    print('Success!! ')
    return df_dup

### Handling NULL values by replacing them

In [ ]:
def handle_NULLs(df,columns):
    print('Replacing NULL values on String Columns with "Unknown" ' , end='')
    df_string = df.fillna('Unknown',subset= columns)
    print('Successs!! ')

    print('Replacing NULL values on Numeric Columns with "0" ' , end='')
    df_clean = df_string.fillna(0,subset = columns)
    print('Success!! ')

    return df_clean

### Getting count of Electric vehicles by creating new column

In [ ]:
def ev_Count(df):
    print('Creating Electric Vehicles Count Column : ', end='')
    from pyspark.sql.functions import col
    df_ev = df.withColumn('Electric_Vehicles_Count',
                            col('EV_Car') + col('EV_Bike')
                            )

    print('Success!! ')
    return df_ev

### Creating columns to get Count of all motor vehicles

In [ ]:
def Motor_Count(df):
    print('Creating All Motor Vehicles Count Column : ', end='')
    from pyspark.sql.functions import col
    df_motor = df.withColumn('Motor_Vehicles_Count',
                            col('Electric_Vehicles_Count') + col('Two_wheeled_motor_vehicles') + col('Cars_and_taxis') + col('Buses_and_coaches') + col('LGV_Type') + col('HGV_Type')
                            )

    print('Success!! ')
    return df_motor

### Creating Transformed Time column

In [ ]:
def create_TransformedTime(df):
    from pyspark.sql.functions import current_timestamp
    print('Creating Transformed Time column : ',end='')
    df_timestamp = df.withColumn('Transformed_Time',
                      current_timestamp()
                      )
    print('Success!!')
    return df_timestamp

### Calling the functions

### Writing the Transformed data to Silver_Traffic Table

In [ ]:
def write_Traffic_SilverTable(StreamingDF,environment):
    print('Writing the silver_traffic Data : ',end='')

    write_StreamSilver = (StreamingDF.writeStream
                .format('delta')
                .option('checkpointLocation',checkpoint+ "/SilverTrafficLoad/Checkpt/")
                .outputMode('append')
                .queryName("SilverTrafficWriteStream")
                .trigger(availableNow=True)
                .toTable(f"`{environment}-catalog`.`silver`.`silver_traffic`"))

    write_StreamSilver.awaitTermination()
    print(f'Writing `{environment}-catalog`.`silver`.`silver_traffic` Success!')

In [ ]:
## Reading the bronze traffic data

df_trafficdata = read_BronzeTrafficTable(env)

# To remove duplicate rows

df_dups = remove_Dups(df_trafficdata)

# To raplce any NULL values
Allcolumns =df_dups.schema.names
df_nulls = handle_NULLs(df_dups,Allcolumns)

## To get the total EV_Count

df_ev = ev_Count(df_nulls)


## To get the Total Motor vehicle count

df_motor = Motor_Count(df_ev)

## Calling Transformed time function

df_final = create_TransformedTime(df_motor)

## Writing to silver_traffic

write_Traffic_SilverTable(df_final, env)

Reading the Bronze Table Data : Reading dev_catalog.bronze.raw_traffic Success!
Removing Duplicate values: Success!! 
Replacing NULL values on String Columns with "Unknown" Successs!! 
Replacing NULL values on Numeric Columns with "0" Success!! 
Creating Electric Vehicles Count Column : Success!! 
Creating All Motor Vehicles Count Column : Success!! 
Creating Transformed Time column : Success!!
Writing the silver_traffic Data : Writing `dev-catalog`.`silver`.`silver_traffic` Success!


### To prove only incremented records were being transformed

###### upload raw_traffic2.csv file into raw_traffic directory in landing directory
###### and upload raw_roads2.csv file into raw_roads directory in landing directory
###### and re run all cells below Part 2b: Calling read and Write Functions
###### You will find different number of counts for below SQL query

###### SIMILARLY

%md
###### upload raw_traffic3.csv file into raw_traffic directory in landing directory
###### and upload raw_roads3.csv file into raw_roads directory in landing directory
###### and re run all cells below Part 2b: Calling read and Write Functions
###### You will find different number of counts for below SQL query

In [ ]:
%sql
select count(*) from `dev-catalog`.`silver`.`silver_traffic`

count(1)
55638


##### Now create the common notebook in another notebook sepeartely with name "Part 2 Common.ipynb"
